In [ ]:
## before anything you need to visit the site below and make sure you have a google earth engine account
## this is so you can access Sentinel-1 GRD and Sentinel-2 TOA and SR products, as well as other sensor packages and data types

## visit the below website below to setup an earth engine account, enable a cloud project, and enable the ee API 
## https://developers.google.com/earth-engine/cloud/earthengine_cloud_project_setup#get-access-to-earth-engine

In [1]:
import ee
import geemap
import math

In [ ]:
## only need to run this once
## after authenticating with google earth engine you will only need to initialize each session

## https://developers.google.com/earth-engine/guides/auth
ee.Authenticate()

In [2]:
## init ee cloud project you made during initial setup
ee.Initialize(project = 'ee-claycaldgsl') ##enter your project name here as a string to initialize exchanges with ee api

# Functions

In [33]:
## Function to add RGB images to the map.
def add_rgb_to_map(image, map_object):

    date = ee.Date(image.get('date')).format('YYYY-MM-dd').getInfo()
    map_object.addLayer(image, {'min': 0, 'max': 2000, 'bands': ['B4', 'B3', 'B2']}, f'{date}_rgb')

## Function to add spectral indices images to the map.
def add_ind_to_map(image, map_object, band):

    date = ee.Date(image.get('date')).format('YYYY-MM-dd').getInfo()
    if band =='NDWI':
        map_object.addLayer(image, {'min': -1, 'max': 1, 'bands': band, 'palette': cm.palettes.ndwi}, f'{date}_{band}')
    elif band =='NDVI': 
        map_object.addLayer(image, {'min': -1, 'max': 1, 'bands': band, 'palette': cm.palettes.ndvi}, f'{date}_{band}')
    elif band == 'MSAVI2':
        map_object.addLayer(image, {'min': -1, 'max': 1, 'bands': band, 'palette': cm.palettes.RdYlGn}, f'{date}_{band}')
    elif band == 'BSI':
        map_object.addLayer(image, {'min': -1, 'max': 1, 'bands': band, 'palette': cm.palettes.Greens}, f'{date}_{band}')

## Function to add spectral indices images to the map.
def add_sar_to_map(image, map_object, target_band):

    date = ee.Date(image.get('date')).format('YYYY-MM-dd').getInfo()
    map_object.addLayer(image, {'min': -50, 'max': 1, 'bands': target_band}, f'{date}_{target_band}')


## functiont to create three important and popular spectral indices
## ndvi = Normalized Difference Vegetation Index, good for vegetation health and cover
## ndwi = Normalized Difference Water Index, good for identifying water bodies and mositure in surface
def s2_10m_target_indices(image):
    # Calculate NDVI
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    # Calculate NDWI
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    # Calculate MSAVI2
    msavi2 = image.expression(
        '((2 * NIR + 1) - ((2 * NIR + 1) ** 2 - 8 * (NIR - RED)) ** 0.5) / 2',
        {
            'NIR': image.select('B8'),
            'RED': image.select('B4')
        }
    ).toFloat().rename('MSAVI2')
    
    # Resample the SWIR1 band to match the resolution of the other bands (10 meters)
    # swir1_resampled = image.select('B11').reproject(crs = msavi2.projection(), scale = 10)
    
    # Calculate the Bare-Soil Index (BSI)
    bsi = image.expression(
        '((SWIR1 + RED) - (NIR + BLUE)) / ((SWIR1 + RED) + (NIR + BLUE))',
        {
            'RED': image.select('B4'),
            'NIR': image.select('B8'),
            'BLUE': image.select('B2'),
            'SWIR1': image.select('B11').resample(mode = 'bicubic') ### scale of 10m specified when exporting images
        }
    ).rename('BSI')
    
    # Add all indices as new bands to the image
    return image.addBands([ndvi, ndwi, msavi2, bsi])

## collects sentinel-1 GRD (radar, no phase) and Sentinel-2 SR (multispectral, adjusted for top of atmosphere reflectance)
def get_sentinel_imagery(aoi, start_date, end_date, s2_cloud_cov, orbit):
    ## Sentinel-1 ImageCollection
    s1 = (ee.ImageCollection('COPERNICUS/S1_GRD')
               .filterBounds(aoi)
               .filterDate(ee.Date(start_date), ee.Date(end_date))
               .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
               .filter(ee.Filter.eq('orbitProperties_pass', orbit))
               .select(['VV', 'VH'])
               .sort('date')
    )

    s1= s1.map(lambda img: img.clip(aoi))
    ## Sentinel-2 Surface Reflectance Harmonized ImageCollection
    s2_10m = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
               .filterBounds(aoi)
               .filterDate(ee.Date(start_date), ee.Date(end_date))
               .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
               .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', s2_cloud_cov))
               .sort('date')
               .select(['B2', 'B3', 'B4', 'B8', 'B11'])
    )
    ## Clip all images in the collection to the AOI
    s2_10m = s2_10m.map(lambda img: img.clip(aoi))
    ## Apply indices to the Sentinel-2 images
    s2_10m_indices = s2_10m.map(s2_10m_target_indices).select(['NDVI', 'NDWI', 'MSAVI2', 'BSI'])
    # s2_10m_ndvi = s2_10m.map(s2_10m_target_indices).select(['NDVI'])
    # s2_10m_ndwi = s2_10m.map(s2_10m_target_indices).select(['NDWI'])
    # s2_10m_msavi2 = s2_10m.map(s2_10m_target_indices).select(['MSAVI2'])
    # s2_10m_bsi = s2_10m.map(s2_10m_target_indices).select(['BSI'])

    
    # return s1_VV, s1_VH, s2_10m, s2_10m_ndvi, s2_10m_ndwi, s2_10m_msavi2, s2_10m_bsi
    return s1, s2_10m, s2_10m_indices #s2_10m_ndvi, s2_10m_ndwi, s2_10m_msavi2, s2_10m_bsi

## fucntion to get the date of each image in the image collection
def get_date(image):
    return ee.Feature(None, {'date': image.date().format('YYYY-MM-dd')})

def glcm_texture_selected(image):

    # image=image.multiply(10000).toInt32() # changes VV and VH to int32, maybe don't need this it adds additonal data manipulation that isnt needed

    # # Compute GLCM texture measures with a window size (e.g., 4 pixels)


    # Convert VV and VH bands from dB to linear scale
    linear_vv = ee.Image.constant(10).pow(image.select('VV').divide(10))
    linear_vh = ee.Image.constant(10).pow(image.select('VH').divide(10))
    
    # Optionally scale the linear values to an integer range for GLCM calculation
    scaled_vv = linear_vv.multiply(10000).toInt32().rename('VV')
    scaled_vh = linear_vh.multiply(10000).toInt32().rename('VH')
    
    # Combine the VV and VH bands into one image
    image_int = scaled_vv.addBands(scaled_vh)


    texture = image_int.select(['VV', 'VH']).glcmTexture(size=3)


    # Add these metrics as new bands to the original image
    return image.addBands(texture)

def export_image_to_drive(collection, collname, image, index):
    # Define the description for the export, incorporating the index for uniqueness

    if collname == 's1':
        description = f"s1_{index}"
    elif collname == 's1_glcm':
        description = f's1_glcm_{index}'
    elif collname == 'rgb':
        description = f's2_10m_{index}'
    elif collname == 'indices':
        description = f's2_10m_{index}'
    elif collname == 'ndvi':
        description = f's2_10m_ndvi_{index}'
    else:
        description = f's2_glcm_{index}'

    # Setup the export task
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=description,
        region=aoi,  # Make sure the geometry is defined earlier
        fileFormat='GeoTIFF',
        scale = 10
    )
    task.start()
    print(f'Exporting {description} to Drive...')

def export_all_images(collection, collname):
    image_list = collection.toList(collection.size())  # Convert ImageCollection to List
    num_images = image_list.size().getInfo()  # Get the number of images

    if collname[:2] == 's1':
        for i, date in enumerate(s1_date_list):
            image = ee.Image(image_list.get(i))
            export_image_to_drive(collection, collname, image, date[:10])

    else:
        for i, date in enumerate(s2_date_list):
            image = ee.Image(image_list.get(i))
            export_image_to_drive(collection, collname, image, date[:10])

def get_s2_glcm(image):

    texture = image.glcmTexture(size=5)


    # Add these metrics as new bands to the original image
    return image.addBands(texture)

In [4]:
# ---------------------------------------------------------------------------//
# 1.SPECKLE FILTERS
# ---------------------------------------------------------------------------//

def boxcar(image, KERNEL_SIZE):
    """
    Apply boxcar filter on every image in the collection.

    Parameters
    ----------
    image : ee.Image
        Image to be filtered
    KERNEL_SIZE : positive odd integer
        Neighbourhood window size

    Returns
    -------
    ee.Image
        Filtered Image

    """
    bandNames = image.bandNames().remove('angle')
      #Define a boxcar kernel
    kernel = ee.Kernel.square((KERNEL_SIZE/2), units='pixels', normalize=True)
     #Apply boxcar
    output = image.select(bandNames).convolve(kernel).rename(bandNames)
    return image.addBands(output, None, True)

def leefilter(image, KERNEL_SIZE):
    """
    Lee Filter applied to one image.
    It is implemented as described in
    J. S. Lee, “Digital image enhancement and noise filtering by use of local statistics,”
    IEEE Pattern Anal. Machine Intell., vol. PAMI-2, pp. 165–168, Mar. 1980.

    Parameters
    ----------
    image : ee.Image
        Image to be filtered
    KERNEL_SIZE : positive odd integer
        Neighbourhood window size

    Returns
    -------
    ee.Image
        Filtered Image

    """
    bandNames = image.bandNames().remove('angle')

    # S1-GRD images are multilooked 5 times in range
    enl = 5
    # Compute the speckle standard deviation
    eta = 1.0/math.sqrt(enl)
    eta = ee.Image.constant(eta)

    # MMSE estimator
    # Neighbourhood mean and variance
    oneImg = ee.Image.constant(1)
    # Estimate stats
    reducers = ee.Reducer.mean().combine(
                      reducer2= ee.Reducer.variance()
                      ,sharedInputs= True
                      )
    stats = (image.select(bandNames).reduceNeighborhood(
                      reducer= reducers
                          ,kernel= ee.Kernel.square(KERNEL_SIZE/2, 'pixels')
                              ,optimization= 'window'))
    meanBand = bandNames.map(lambda bandName: ee.String(bandName).cat('_mean'))
    varBand = bandNames.map(lambda bandName:  ee.String(bandName).cat('_variance'))

    z_bar = stats.select(meanBand)
    varz = stats.select(varBand)
    # Estimate weight 
    varx = (varz.subtract(z_bar.pow(2).multiply(eta.pow(2)))).divide(oneImg.add(eta.pow(2)))
    b = varx.divide(varz)
  
    # if b is negative set it to zero
    new_b = b.where(b.lt(0), 0)
    output = oneImg.subtract(new_b).multiply(z_bar.abs()).add(new_b.multiply(image.select(bandNames)))
    output = output.rename(bandNames)
    return image.addBands(output, None, True)


def gammamap(image,KERNEL_SIZE): 
    
    """
    Gamma Maximum a-posterior Filter applied to one image. It is implemented as described in 
    Lopes A., Nezry, E., Touzi, R., and Laur, H., 1990.  
    Maximum A Posteriori Speckle Filtering and First Order texture Models in SAR Images.  
    International  Geoscience  and  Remote  Sensing  Symposium (IGARSS).
    Parameters
    ----------
    image : ee.Image
        Image to be filtered
    KERNEL_SIZE : positive odd integer
        Neighbourhood window size
    Returns
    -------
    ee.Image
        Filtered Image
    """
    enl = 5
    bandNames = image.bandNames().remove('angle')
    #local mean
    reducers = ee.Reducer.mean().combine( \
                      reducer2= ee.Reducer.stdDev(), \
                      sharedInputs= True
                      )
    stats = (image.select(bandNames).reduceNeighborhood( \
                      reducer= reducers, \
                          kernel= ee.Kernel.square(KERNEL_SIZE/2,'pixels'), \
                              optimization= 'window'))
    meanBand = bandNames.map(lambda bandName: ee.String(bandName).cat('_mean'))
    stdDevBand = bandNames.map(lambda bandName:  ee.String(bandName).cat('_stdDev'))
        
    z = stats.select(meanBand)
    sigz = stats.select(stdDevBand)
    
    #local observed coefficient of variation
    ci = sigz.divide(z)
    #noise coefficient of variation (or noise sigma)
    cu = 1.0/math.sqrt(enl)
    #threshold for the observed coefficient of variation
    cmax = math.sqrt(2.0) * cu
    cu = ee.Image.constant(cu)
    cmax = ee.Image.constant(cmax)
    enlImg = ee.Image.constant(enl)
    oneImg = ee.Image.constant(1)
    twoImg = ee.Image.constant(2)

    alpha = oneImg.add(cu.pow(2)).divide(ci.pow(2).subtract(cu.pow(2)))

    #Implements the Gamma MAP filter described in equation 11 in Lopez et al. 1990
    q = image.select(bandNames).expression('z**2 * (z * alpha - enl - 1)**2 + 4 * alpha * enl * b() * z', { 'z': z,  'alpha':alpha,'enl': enl})
    rHat = z.multiply(alpha.subtract(enlImg).subtract(oneImg)).add(q.sqrt()).divide(twoImg.multiply(alpha))
  
    #if ci <= cu then its a homogenous region ->> boxcar filter
    zHat = (z.updateMask(ci.lte(cu))).rename(bandNames)
    #if cmax > ci > cu then its a textured medium ->> apply Gamma MAP filter
    rHat = (rHat.updateMask(ci.gt(cu)).updateMask(ci.lt(cmax))).rename(bandNames)
    #ci>cmax then its strong signal ->> retain
    x = image.select(bandNames).updateMask(ci.gte(cmax)).rename(bandNames)  
    #Merge
    output = ee.ImageCollection([zHat,rHat,x]).sum()
    return image.addBands(output, None, True)

def RefinedLee(image):
    """
    This filter is modified from the implementation by Guido Lemoine 
    Source: Lemoine et al. https://code.earthengine.google.com/5d1ed0a0f0417f098fdfd2fa137c3d0c

    Parameters
    ----------
    image: ee.Image
        Image to be filtered

    Returns
    -------
    result: ee.Image
        Filtered Image

    """

    bandNames = image.bandNames().remove('angle')

    def inner(b):

        img = image.select([b]);
    
        # img must be linear, i.e. not in dB!
        # Set up 3x3 kernels 
        weights3 = ee.List.repeat(ee.List.repeat(1,3),3);
        kernel3 = ee.Kernel.fixed(3,3, weights3, 1, 1, False);
  
        mean3 = img.reduceNeighborhood(ee.Reducer.mean(), kernel3);
        variance3 = img.reduceNeighborhood(ee.Reducer.variance(), kernel3);
  
        # Use a sample of the 3x3 windows inside a 7x7 windows to determine gradients and directions
        sample_weights = ee.List([[0,0,0,0,0,0,0], [0,1,0,1,0,1,0],[0,0,0,0,0,0,0], [0,1,0,1,0,1,0], [0,0,0,0,0,0,0], [0,1,0,1,0,1,0],[0,0,0,0,0,0,0]]);
  
        sample_kernel = ee.Kernel.fixed(7,7, sample_weights, 3,3, False);
  
        # Calculate mean and variance for the sampled windows and store as 9 bands
        sample_mean = mean3.neighborhoodToBands(sample_kernel); 
        sample_var = variance3.neighborhoodToBands(sample_kernel);
  
        # Determine the 4 gradients for the sampled windows
        gradients = sample_mean.select(1).subtract(sample_mean.select(7)).abs();
        gradients = gradients.addBands(sample_mean.select(6).subtract(sample_mean.select(2)).abs());
        gradients = gradients.addBands(sample_mean.select(3).subtract(sample_mean.select(5)).abs());
        gradients = gradients.addBands(sample_mean.select(0).subtract(sample_mean.select(8)).abs());
  
        # And find the maximum gradient amongst gradient bands
        max_gradient = gradients.reduce(ee.Reducer.max());
  
        # Create a mask for band pixels that are the maximum gradient
        gradmask = gradients.eq(max_gradient);
  
        # duplicate gradmask bands: each gradient represents 2 directions
        gradmask = gradmask.addBands(gradmask);
  
        # Determine the 8 directions
        directions = sample_mean.select(1).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(7))).multiply(1);
        directions = directions.addBands(sample_mean.select(6).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(2))).multiply(2));
        directions = directions.addBands(sample_mean.select(3).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(5))).multiply(3));
        directions = directions.addBands(sample_mean.select(0).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(8))).multiply(4));
        # The next 4 are the not() of the previous 4
        directions = directions.addBands(directions.select(0).Not().multiply(5));
        directions = directions.addBands(directions.select(1).Not().multiply(6));
        directions = directions.addBands(directions.select(2).Not().multiply(7));
        directions = directions.addBands(directions.select(3).Not().multiply(8));
  
        # Mask all values that are not 1-8
        directions = directions.updateMask(gradmask);
  
        # "collapse" the stack into a singe band image (due to masking, each pixel has just one value (1-8) in it's directional band, and is otherwise masked)
        directions = directions.reduce(ee.Reducer.sum());  
  
        sample_stats = sample_var.divide(sample_mean.multiply(sample_mean));
  
        #Calculate localNoiseVariance
        sigmaV = sample_stats.toArray().arraySort().arraySlice(0,0,5).arrayReduce(ee.Reducer.mean(), [0]);
  
        # Set up the 7*7 kernels for directional statistics
        rect_weights = ee.List.repeat(ee.List.repeat(0,7),3).cat(ee.List.repeat(ee.List.repeat(1,7),4));
  
        diag_weights = ee.List([[1,0,0,0,0,0,0], [1,1,0,0,0,0,0], [1,1,1,0,0,0,0], [1,1,1,1,0,0,0], [1,1,1,1,1,0,0], [1,1,1,1,1,1,0], [1,1,1,1,1,1,1]]);
  
        rect_kernel = ee.Kernel.fixed(7,7, rect_weights, 3, 3, False);
        diag_kernel = ee.Kernel.fixed(7,7, diag_weights, 3, 3, False);
  
        # Create stacks for mean and variance using the original kernels. Mask with relevant direction.
        dir_mean = img.reduceNeighborhood(ee.Reducer.mean(), rect_kernel).updateMask(directions.eq(1));
        dir_var = img.reduceNeighborhood(ee.Reducer.variance(), rect_kernel).updateMask(directions.eq(1));
  
        dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), diag_kernel).updateMask(directions.eq(2)));
        dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), diag_kernel).updateMask(directions.eq(2)));
  
        # and add the bands for rotated kernels
        for i in range(1, 4):
            dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), rect_kernel.rotate(i)).updateMask(directions.eq(2*i+1)))
            dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), rect_kernel.rotate(i)).updateMask(directions.eq(2*i+1)))
            dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), diag_kernel.rotate(i)).updateMask(directions.eq(2*i+2)))
            dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), diag_kernel.rotate(i)).updateMask(directions.eq(2*i+2)))

  
        # "collapse" the stack into a single band image (due to masking, each pixel has just one value in it's directional band, and is otherwise masked)
        dir_mean = dir_mean.reduce(ee.Reducer.sum());
        dir_var = dir_var.reduce(ee.Reducer.sum());
  
        # A finally generate the filtered value
        varX = dir_var.subtract(dir_mean.multiply(dir_mean).multiply(sigmaV)).divide(sigmaV.add(1.0))
  
        b = varX.divide(dir_var)
        result = dir_mean.add(b.multiply(img.subtract(dir_mean)))
  
        return result.arrayProject([0]).arrayFlatten([['sum']]).float()
    
    result = ee.ImageCollection(bandNames.map(inner)).toBands().rename(bandNames).copyProperties(image)
    
    return image.addBands(result, None, True) 



def leesigma(image,KERNEL_SIZE):
    """
    Implements the improved lee sigma filter to one image. 
    It is implemented as described in, Lee, J.-S. Wen, J.-H. Ainsworth, T.L. Chen, K.-S. Chen, A.J. 
    Improved sigma filter for speckle filtering of SAR imagery. 
    IEEE Trans. Geosci. Remote Sens. 2009, 47, 202–213.

    Parameters
    ----------
    image : ee.Image
        Image to be filtered
    KERNEL_SIZE : positive odd integer
        Neighbourhood window size

    Returns
    -------
    ee.Image
        Filtered Image

    """

    #parameters
    Tk = ee.Image.constant(7) #number of bright pixels in a 3x3 window
    sigma = 0.9
    enl = 4
    target_kernel = 3
    bandNames = image.bandNames().remove('angle')
  
    #compute the 98 percentile intensity 
    z98 = ee.Dictionary(image.select(bandNames).reduceRegion(
                reducer= ee.Reducer.percentile([98]),
                geometry= image.geometry(),
                scale=10,
                maxPixels=1e13
            )).toImage()
  

    #select the strong scatterers to retain
    brightPixel = image.select(bandNames).gte(z98)
    K = brightPixel.reduceNeighborhood(ee.Reducer.countDistinctNonNull()
            ,ee.Kernel.square(target_kernel/2)) 
    retainPixel = K.gte(Tk)
  
  
    #compute the a-priori mean within a 3x3 local window
    #original noise standard deviation since the data is 5 look
    eta = 1.0/math.sqrt(enl) 
    eta = ee.Image.constant(eta)
    #MMSE applied to estimate the apriori mean
    reducers = ee.Reducer.mean().combine( \
                      reducer2= ee.Reducer.variance(), \
                      sharedInputs= True
                      )
    stats = image.select(bandNames).reduceNeighborhood( \
                      reducer= reducers, \
                          kernel= ee.Kernel.square(target_kernel/2,'pixels'), \
                              optimization= 'window')
    meanBand = bandNames.map(lambda bandName: ee.String(bandName).cat('_mean'))
    varBand = bandNames.map(lambda bandName:  ee.String(bandName).cat('_variance'))
        
    z_bar = stats.select(meanBand)
    varz = stats.select(varBand)
    
    oneImg = ee.Image.constant(1)
    varx = (varz.subtract(z_bar.abs().pow(2).multiply(eta.pow(2)))).divide(oneImg.add(eta.pow(2)))
    b = varx.divide(varz)
    xTilde = oneImg.subtract(b).multiply(z_bar.abs()).add(b.multiply(image.select(bandNames)))
  
    #step 3: compute the sigma range
    #Lookup table (J.S.Lee et al 2009) for range and eta values for intensity (only 4 look is shown here)
    LUT = ee.Dictionary({0.5: ee.Dictionary({'I1': 0.694,'I2': 1.385,'eta': 0.1921}),
                                 0.6: ee.Dictionary({'I1': 0.630,'I2': 1.495,'eta': 0.2348}),
                                 0.7: ee.Dictionary({'I1': 0.560,'I2': 1.627,'eta': 0.2825}),
                                 0.8: ee.Dictionary({'I1': 0.480,'I2': 1.804,'eta': 0.3354}),
                                 0.9: ee.Dictionary({'I1': 0.378,'I2': 2.094,'eta': 0.3991}),
                                 0.95: ee.Dictionary({'I1': 0.302,'I2': 2.360,'eta': 0.4391})});
  
    #extract data from lookup
    sigmaImage = ee.Dictionary(LUT.get(str(sigma))).toImage()
    I1 = sigmaImage.select('I1')
    I2 = sigmaImage.select('I2')
    #new speckle sigma
    nEta = sigmaImage.select('eta')
    #establish the sigma ranges
    I1 = I1.multiply(xTilde)
    I2 = I2.multiply(xTilde)
  
    #step 3: apply MMSE filter for pixels in the sigma range
    #MMSE estimator
    mask = image.select(bandNames).gte(I1).Or(image.select(bandNames).lte(I2))
    z = image.select(bandNames).updateMask(mask)
  
    stats = z.reduceNeighborhood( \
                      reducer= reducers, \
                          kernel= ee.Kernel.square(KERNEL_SIZE/2,'pixels'), \
                              optimization= 'window')
        
    z_bar = stats.select(meanBand)
    varz = stats.select(varBand)
    
    
    varx = (varz.subtract(z_bar.abs().pow(2).multiply(nEta.pow(2)))).divide(oneImg.add(nEta.pow(2)))
    b = varx.divide(varz)
    #if b is negative set it to zero
    new_b = b.where(b.lt(0), 0)
    xHat = oneImg.subtract(new_b).multiply(z_bar.abs()).add(new_b.multiply(z))
  
    #remove the applied masks and merge the retained pixels and the filtered pixels
    xHat = image.select(bandNames).updateMask(retainPixel).unmask(xHat)
    output = ee.Image(xHat).rename(bandNames)
    return image.addBands(output, None, True)


#---------------------------------------------------------------------------//
# 2. MONO-TEMPORAL SPECKLE FILTER (WRAPPER)
#---------------------------------------------------------------------------//


def MonoTemporal_Filter(coll,KERNEL_SIZE, SPECKLE_FILTER) :
    """
    A wrapper function for monotemporal filter

    Parameters
    ----------
    coll : ee Image collection
        the image collection to be filtered
    KERNEL_SIZE : odd integer
        Spatial Neighbourhood window
    SPECKLE_FILTER : String
        Type of speckle filter

    Returns
    -------
    ee.ImageCollection
        An image collection where a mono-temporal filter is applied to each 
        image individually

    """
    def _filter(image):    
       if (SPECKLE_FILTER=='BOXCAR'):
          _filtered = boxcar(image, KERNEL_SIZE)
       elif (SPECKLE_FILTER=='LEE'):
          _filtered = leefilter(image, KERNEL_SIZE)
       elif (SPECKLE_FILTER=='GAMMA MAP'):
          _filtered = gammamap(image, KERNEL_SIZE)
       elif (SPECKLE_FILTER=='REFINED LEE'):
          _filtered = RefinedLee(image)
       elif (SPECKLE_FILTER=='LEE SIGMA'):
          _filtered = leesigma(image, KERNEL_SIZE)
       return _filtered
    return coll.map(_filter)

# ---------------------------------------------------------------------------//
# 3. MULTI-TEMPORAL SPECKLE FILTER
# ---------------------------------------------------------------------------//

def MultiTemporal_Filter(coll,KERNEL_SIZE, SPECKLE_FILTER,NR_OF_IMAGES):
    """

    A wrapper function for multi-temporal filter

    Parameters
    ----------
    coll : ee Image collection
        the image collection to be filtered
    KERNEL_SIZE : odd integer
        Spatial Neighbourhood window
    SPECKLE_FILTER : String
        Type of speckle filter
    NR_OF_IMAGES : positive integer
        Number of images to use in multi-temporal filtering

    Returns
    -------
    ee.ImageCollection
        An image collection where a multi-temporal filter is applied to each
        image individually

    """
  
    def Quegan(image) :
        """
        The following Multi-temporal speckle filters are implemented as described in
        S. Quegan and J. J. Yu, “Filtering of multichannel SAR images,” 
        IEEE Trans Geosci. Remote Sensing, vol. 39, Nov. 2001.
        
        this function will filter the collection used for the multi-temporal part
        it takes care of:
        - same image geometry (i.e relative orbit)
        - full overlap of image
        - amount of images taken for filtering 
            -- all before
           -- if not enough, images taken after the image to filter are added

        Parameters
        ----------
        image : ee.Image
            Image to be filtered

        Returns
        -------
        ee.Image
            Filtered image

        """
        def setresample(image):
                return image.resample()
            
        def get_filtered_collection(image):
            """
            Generate a dedicated image collection

            Parameters
            ----------
            image : ee.Image
                Image whose geometry is used to define the new collection

            Returns
            -------
            ee Image collection

            """
  
            #filter collection over are and by relative orbit
            s1_coll = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT') \
                .filterBounds(image.geometry()) \
                .filter(ee.Filter.eq('instrumentMode', 'IW')) \
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', ee.List(image.get('transmitterReceiverPolarisation')).get(-1))) \
                .filter(ee.Filter.Or(ee.Filter.eq('relativeOrbitNumber_stop', image.get('relativeOrbitNumber_stop')), \
                                     ee.Filter.eq('relativeOrbitNumber_stop', image.get('relativeOrbitNumber_start'))
                )).map(setresample)
      
            #a function that takes the image and checks for the overlap
            def check_overlap(_image):
                """
                get all S1 frames from this date intersecting with the image bounds

                Parameters
                ----------
                _image : ee.Image
                    Image to check the overlap with

                Returns
                -------
                ee Image Collection
                    A collection with matching geometry

                """
                
                # get all S1 frames from this date intersecting with the image bounds
                s1 = s1_coll.filterDate(_image.date(), _image.date().advance(1, 'day'))
                # intersect those images with the image to filter
                intersect = image.geometry().intersection(s1.geometry().dissolve(), 10)
                # check if intersect is sufficient
                valid_date = ee.Algorithms.If(intersect.area(10).divide(image.geometry().area(10)).gt(0.95), \
                                              _image.date().format('YYYY-MM-dd')
                                              )
                return ee.Feature(None, {'date': valid_date})
      
      
            # this function will pick up the acq dates for fully overlapping acquisitions before the image acquistion
            dates_before = s1_coll.filterDate('2014-01-01', image.date().advance(1, 'day')) \
                                    .sort('system:time_start', False).limit(5*NR_OF_IMAGES) \
                                    .map(check_overlap).distinct('date').aggregate_array('date')
    
            # if the images before are not enough, we add images from after the image acquisition 
            # this will only be the case at the beginning of S1 mission
            dates = ee.List(ee.Algorithms.If( \
                                             dates_before.size().gte(NR_OF_IMAGES), \
                                                 dates_before.slice(0, NR_OF_IMAGES), \
                                                     s1_coll \
                                                         .filterDate(image.date(), '2100-01-01') \
                                                             .sort('system:time_start', True).limit(5*NR_OF_IMAGES) \
                                                                 .map(check_overlap) \
                                                                     .distinct('date') \
                                                                         .aggregate_array('date') \
                                                                             .cat(dates_before).distinct().sort().slice(0, NR_OF_IMAGES)
                                                                             )
                                                )
    
            #now we re-filter the collection to get the right acquisitions for multi-temporal filtering
            return ee.ImageCollection(dates.map(lambda date: s1_coll.filterDate(date, ee.Date(date).advance(1,'day')).toList(s1_coll.size())).flatten())
      
          
  
        #we get our dedicated image collection for that image
        s1 = get_filtered_collection(image)
  
        bands = image.bandNames().remove('angle')
        s1 = s1.select(bands)
        meanBands = bands.map(lambda bandName: ee.String(bandName).cat('_mean'))
        ratioBands = bands.map(lambda bandName: ee.String(bandName).cat('_ratio'))
        count_img = s1.reduce(ee.Reducer.count())

        def inner(image):
            """
            Creats an image whose bands are the filtered image and image ratio

            Parameters
            ----------
            image : ee.Image
                Image to be filtered

            Returns
            -------
            ee.Image
                Filtered image and image ratio

            """
            if (SPECKLE_FILTER=='BOXCAR'):
                _filtered = boxcar(image, KERNEL_SIZE).select(bands).rename(meanBands) 
            elif (SPECKLE_FILTER=='LEE'):
                _filtered = leefilter(image, KERNEL_SIZE).select(bands).rename(meanBands)
            elif (SPECKLE_FILTER=='GAMMA MAP'):
                _filtered = gammamap(image, KERNEL_SIZE).select(bands).rename(meanBands)
            elif (SPECKLE_FILTER=='REFINED LEE'):
                _filtered = RefinedLee(image).select(bands).rename(meanBands)
            elif (SPECKLE_FILTER=='LEE SIGMA'):
                _filtered = leesigma(image, KERNEL_SIZE).select(bands).rename(meanBands)
    
            _ratio = image.select(bands).divide(_filtered).rename(ratioBands) 
            return _filtered.addBands(_ratio)

        isum = s1.map(inner).select(ratioBands).reduce(ee.Reducer.sum())
        filtered = inner(image).select(meanBands)
        divide = filtered.divide(count_img)
        output = divide.multiply(isum).rename(bands)

        return image.addBands(output, None, True)
    return coll.map(Quegan)

# Get area of interest
1. Sabine National Wildlife Refuge (Louisiana) -- Gulf Coast
2. Deer Island (Mississippi) -- Gulf Coast
3. SMIIL (New Jersey) -- East Coast
4. Poplar Island (Maryland) -- East Coast
5. South Bay Salt Pond Restoration (California) -- West Coast
6. Niaqually National Wildlife Refuge (Washington) -- West Coast
7. Kachemak Bay National Estuarine Research Researve (Alaska) -- Arctic 
8. Mendenhall Wetlands (Alaska) -- Arctic

Going to look for sites on the EWN as well

In [21]:
## interactive map for you to draw a polygon to signify your aoi

## Create a map centered at a specific location
m = geemap.Map(center=[20, 0], zoom=2, basemap='HYBRID')
## Add drawing tools
m.add_draw_control()
## Display the map
display(m)

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(child…

In [22]:
## Get the drawn features
draw_features = m.draw_features[0]
## Establish ee.Polygon from drawn area of interest to collect imagery
aoi = ee.Geometry.Polygon(draw_features.getInfo()['geometry']['coordinates'][0])

# Get Imagery

In [23]:
start_date = '2019-10-01' ## start date of search window
end_date = '2024-11-06' ## end date of search window
s2_cloud_cov = 1 ## percentage of clouds in sentinel-2 multispectral imagery, less means you see more surface
orbit = 'ASCENDING' ## orbit for imagery

collections = {}
for i in range(int(start_date[:4]), int(end_date[:4])):
    # collections[f's1_{i}_{i+1}'], collections[f's2_10m_{i}_{i+1}'], collections[f's2_10m_ndvi_{i}_{i+1}'], collections[f's2_10m_ndwi_{i}_{i+1}'], collections[f's2_10m_msavi2_{i}_{i+1}'], collections[f's2_10m_bsi_{i}_{i+1}']  = get_sentinel_imagery(aoi, f'{i}{start_date[4:]}', f'{i+1}{end_date[4:]}', s2_cloud_cov, orbit)
    collections[f's1_{i}_{i+1}'], collections[f's2_10m_{i}_{i+1}'], collections[f's2_10m_indices_{i}_{i+1}'] = get_sentinel_imagery(aoi, f'{i}{start_date[4:]}', f'{i+1}{end_date[4:]}', s2_cloud_cov, orbit)

In [24]:
total_colls = {}
# total_colls['s1'], total_colls['s2_10m'], total_colls['s2_10m_ndvi'], total_colls['s2_10m_ndwi'], total_colls['s2_10m_msavi2'], total_colls['s2_10m_bsi']= get_sentinel_imagery(aoi, start_date, end_date, s2_cloud_cov, orbit)
total_colls['s1'], total_colls['s2_10m'], total_colls['s2_10m_indices']= get_sentinel_imagery(aoi, start_date, end_date, s2_cloud_cov, orbit)

In [25]:
colls = []
for i, coll in enumerate(collections):
    if collections[coll].size().getInfo() != 0: # removes image collections that are empty
        colls.append(coll)

for i, coll in enumerate(colls):
    print(f'{i}: {coll}')

0: s1_2019_2020
1: s2_10m_2019_2020
2: s2_10m_indices_2019_2020
3: s1_2020_2021
4: s2_10m_2020_2021
5: s2_10m_indices_2020_2021
6: s1_2021_2022
7: s2_10m_2021_2022
8: s2_10m_indices_2021_2022
9: s1_2022_2023
10: s2_10m_2022_2023
11: s2_10m_indices_2022_2023
12: s1_2023_2024
13: s2_10m_2023_2024
14: s2_10m_indices_2023_2024


# Apply speckle filtering to the Sentinel-1 Imagery

In [27]:
# Monotemporal
# filters based on images of a single image
s1monofilt = MonoTemporal_Filter(total_colls['s1'], 5, 'REFINED LEE')

# Multitemporal
# filters based on the pixels in each image of the entire image collection
# s1multifilt = MultiTemporal_Filter(total_colls['s1'], 3, 'GAMMA MAP', total_colls['s1'].size().getInfo())

# Get GLCM Texture Features for Setninel-1 VV and VH bands

In [28]:
s1_glcm_bands = ['VV_contrast', 'VV_diss', 'VV_corr', 'VV_ent',
                 'VH_contrast', 'VH_diss', 'VH_corr', 'VH_ent']

# Apply the function to the im collection
# s1_glcm = total_colls['s1'].map(glcm_texture_selected).select(s1_glcm_bands)
s1_filt_glcm = s1monofilt.map(glcm_texture_selected).select(s1_glcm_bands)

In [29]:
glcmbands = ['_contrast', '_diss', '_corr', '_ent']

targetglcm ={}
s2glcmcolls = {}

for band in total_colls['s2_10m'].first().bandNames().getInfo():
    targetglcm[band] = [band+glcm for glcm in glcmbands]
    # s2glcmcolls[band] = total_colls['s2_10m'].map(get_s2_glcm).select([band+glcm for glcm in glcmbands])

In [30]:
s2glcm = total_colls['s2_10m'].map(get_s2_glcm).select([band+glcm for glcm in glcmbands for band in total_colls['s2_10m'].first().bandNames().getInfo()])

In [35]:
s2glcm

Name,Description
B1,Aerosols
B2,Blue
B3,Green
B4,Red
B5,Red Edge 1
B6,Red Edge 2
B7,Red Edge 3
B8,NIR
B8A,Red Edge 4
B9,Water vapor


In [20]:
band = 'B2_var'


Map = geemap.Map()
Map.centerObject(aoi, 12)

# Visualize each image in the ImageCollection.
images = s2glcm.toList(s2glcm.size())
for i in range(s2glcm.size().getInfo()//30):
    image = ee.Image(images.get(-i))
    date = ee.Date(image.get('date')).format('YYYY-MM-dd').getInfo()

    maxval = ee.Image(images.get(0)).select([band]).reduceRegion(
        reducer=ee.Reducer.max(),
        scale=10,  # meters
        maxPixels= 20000000
    ).get(band).getInfo()

    minval = ee.Image(images.get(0)).select([band]).reduceRegion(
        reducer=ee.Reducer.min(),
        scale=10,  # meters
        maxPixels= 20000000
    ).get(band).getInfo()

    Map.addLayer(image, {'min': minval,'max': maxval,'bands': [band]}, f'{date}')

# Display the map.
Map.addLayerControl(position = 'topright')
Map

Map(center=[32.6043845973287, -117.11391449999829], controls=(WidgetControl(options=['position', 'transparent_…

# Map imagery

In [ ]:
####rgb map####

Map = geemap.Map()
Map.centerObject(aoi, 12)

# Visualize each image in the ImageCollection.
s2_images = collections[colls[1]].toList(collections[colls[1]].size())
for i in range(collections[colls[1]].size().getInfo()//10):
    image = ee.Image(s2_images.get(-i))
    add_rgb_to_map(image, Map)

# Display the map.
Map.addLayerControl(position = 'topright')
Map

In [ ]:
Map_sar_vv = geemap.Map()
Map_sar_vv.centerObject(aoi, 12)

# Visualize each image in the ImageCollection.
s1_images = total_colls['s1'].toList(total_colls['s1'].size())
for i in range(total_colls['s1'].size().getInfo()//40):
    image = ee.Image(s1_images.get(i))
    add_sar_to_map(image, Map_sar_vv, 'VV')

# Display the map.
Map_sar_vv.addLayerControl(position = 'topright')
Map_sar_vv

In [ ]:
Map_sar = geemap.Map()
Map_sar.centerObject(aoi, 12)

# Visualize each image in the ImageCollection.
s1_images = s1monofilt.toList(s1monofilt.size())
for i in range(s1monofilt.size().getInfo()//40):
    image = ee.Image(s1_images.get(i))
    add_sar_to_map(image, Map_sar, 'VV')

# Display the map.
Map_sar.addLayerControl(position = 'topright')
Map_sar

In [ ]:
Map_ind = geemap.Map()

# Visualize each image in the ImageCollection.
s2ind_images = collections[colls[3]].toList(collections[colls[3]].size())
for i in range(collections[colls[3]].size().getInfo()//2):
    image = ee.Image(s2ind_images.get(i))
    add_ind_to_map(image, Map_ind, collections[colls[3]])

# Display the map.
Map_ind.addLayerControl(position = 'topright')
Map_ind

In [ ]:
Map_ind = geemap.Map()

# Visualize each image in the ImageCollection.
s2ind_images = collections[colls[4]].toList(collections[colls[4]].size())
for i in range(collections[colls[4]].size().getInfo()//2):
    image = ee.Image(s2ind_images.get(i))
    add_ind_to_map(image, Map_ind, collections[colls[4]])

# Display the map.
Map_ind.addLayerControl(position = 'topright')
Map_ind

# Export imagery to google drive

In [34]:
s2_date_list = total_colls['s2_10m'].map(get_date).aggregate_array('date').getInfo()
export_all_images(total_colls['s2_10m'], 'rgb')

Exporting s2_10m_2019-11-15 to Drive...
Exporting s2_10m_2019-12-30 to Drive...
Exporting s2_10m_2020-01-04 to Drive...
Exporting s2_10m_2020-01-24 to Drive...
Exporting s2_10m_2020-02-28 to Drive...
Exporting s2_10m_2020-09-30 to Drive...
Exporting s2_10m_2020-10-10 to Drive...
Exporting s2_10m_2020-10-30 to Drive...
Exporting s2_10m_2020-11-04 to Drive...
Exporting s2_10m_2020-12-09 to Drive...
Exporting s2_10m_2020-12-29 to Drive...
Exporting s2_10m_2021-01-13 to Drive...
Exporting s2_10m_2021-01-18 to Drive...
Exporting s2_10m_2021-02-07 to Drive...
Exporting s2_10m_2021-02-22 to Drive...
Exporting s2_10m_2021-03-29 to Drive...
Exporting s2_10m_2021-09-10 to Drive...
Exporting s2_10m_2021-10-30 to Drive...
Exporting s2_10m_2021-11-14 to Drive...
Exporting s2_10m_2021-11-19 to Drive...
Exporting s2_10m_2021-11-29 to Drive...
Exporting s2_10m_2022-01-03 to Drive...
Exporting s2_10m_2022-01-13 to Drive...
Exporting s2_10m_2022-03-24 to Drive...
Exporting s2_10m_2022-04-03 to Drive...


In [36]:
s2_date_list = s2glcm.map(get_date).aggregate_array('date').getInfo()
export_all_images(s2glcm, 's2_glcm')

Exporting s2_glcm_2019-11-15 to Drive...
Exporting s2_glcm_2019-12-30 to Drive...
Exporting s2_glcm_2020-01-04 to Drive...
Exporting s2_glcm_2020-01-24 to Drive...
Exporting s2_glcm_2020-02-28 to Drive...
Exporting s2_glcm_2020-09-30 to Drive...
Exporting s2_glcm_2020-10-10 to Drive...
Exporting s2_glcm_2020-10-30 to Drive...
Exporting s2_glcm_2020-11-04 to Drive...
Exporting s2_glcm_2020-12-09 to Drive...
Exporting s2_glcm_2020-12-29 to Drive...
Exporting s2_glcm_2021-01-13 to Drive...
Exporting s2_glcm_2021-01-18 to Drive...
Exporting s2_glcm_2021-02-07 to Drive...
Exporting s2_glcm_2021-02-22 to Drive...
Exporting s2_glcm_2021-03-29 to Drive...
Exporting s2_glcm_2021-09-10 to Drive...
Exporting s2_glcm_2021-10-30 to Drive...
Exporting s2_glcm_2021-11-14 to Drive...
Exporting s2_glcm_2021-11-19 to Drive...
Exporting s2_glcm_2021-11-29 to Drive...
Exporting s2_glcm_2022-01-03 to Drive...
Exporting s2_glcm_2022-01-13 to Drive...
Exporting s2_glcm_2022-03-24 to Drive...
Exporting s2_glc

In [37]:
s1_date_list = s1monofilt.map(get_date).aggregate_array('date').getInfo()
export_all_images(s1monofilt, 's1')

Exporting s1_2019-10-01 to Drive...
Exporting s1_2019-10-13 to Drive...
Exporting s1_2019-11-06 to Drive...
Exporting s1_2019-11-18 to Drive...
Exporting s1_2019-11-30 to Drive...
Exporting s1_2019-12-12 to Drive...
Exporting s1_2020-01-17 to Drive...
Exporting s1_2020-01-29 to Drive...
Exporting s1_2020-02-10 to Drive...
Exporting s1_2020-03-17 to Drive...
Exporting s1_2020-03-29 to Drive...
Exporting s1_2020-04-10 to Drive...
Exporting s1_2020-04-22 to Drive...
Exporting s1_2020-05-04 to Drive...
Exporting s1_2020-05-16 to Drive...
Exporting s1_2020-05-28 to Drive...
Exporting s1_2020-06-09 to Drive...
Exporting s1_2020-06-21 to Drive...
Exporting s1_2020-07-03 to Drive...
Exporting s1_2020-07-15 to Drive...
Exporting s1_2020-07-27 to Drive...
Exporting s1_2020-08-08 to Drive...
Exporting s1_2020-08-20 to Drive...
Exporting s1_2020-09-01 to Drive...
Exporting s1_2020-09-13 to Drive...
Exporting s1_2020-09-25 to Drive...
Exporting s1_2020-10-07 to Drive...
Exporting s1_2020-10-19 to D

In [38]:
s1_date_list = s1_filt_glcm.map(get_date).aggregate_array('date').getInfo()
export_all_images(s1_filt_glcm, 's1_glcm')

Exporting s1_glcm_2019-10-01 to Drive...
Exporting s1_glcm_2019-10-13 to Drive...
Exporting s1_glcm_2019-11-06 to Drive...
Exporting s1_glcm_2019-11-18 to Drive...
Exporting s1_glcm_2019-11-30 to Drive...
Exporting s1_glcm_2019-12-12 to Drive...
Exporting s1_glcm_2020-01-17 to Drive...
Exporting s1_glcm_2020-01-29 to Drive...
Exporting s1_glcm_2020-02-10 to Drive...
Exporting s1_glcm_2020-03-17 to Drive...
Exporting s1_glcm_2020-03-29 to Drive...
Exporting s1_glcm_2020-04-10 to Drive...
Exporting s1_glcm_2020-04-22 to Drive...
Exporting s1_glcm_2020-05-04 to Drive...
Exporting s1_glcm_2020-05-16 to Drive...
Exporting s1_glcm_2020-05-28 to Drive...
Exporting s1_glcm_2020-06-09 to Drive...
Exporting s1_glcm_2020-06-21 to Drive...
Exporting s1_glcm_2020-07-03 to Drive...
Exporting s1_glcm_2020-07-15 to Drive...
Exporting s1_glcm_2020-07-27 to Drive...
Exporting s1_glcm_2020-08-08 to Drive...
Exporting s1_glcm_2020-08-20 to Drive...
Exporting s1_glcm_2020-09-01 to Drive...
Exporting s1_glc